In [2]:
import mysql.connector

config = {
  'user': 'admin',
  'password': 'TradEasy1234567890.',
  'host': 'tradeasyquotation.c9ac6sewqau0.ap-southeast-2.rds.amazonaws.com',
  'database': 'Tradeasy_quotation',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)
# ... perform database operations ...
cnx.close()
